In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score # also score and accuracy 
import statsmodels.api as sm 

from statsmodels.stats.outliers_influence import variance_inflation_factor

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

**SourceCode**

https://www.kaggle.com/code/tanavbajaj/sonar-rock-vs-mine-classifier

**DataSource**

https://www.kaggle.com/datasets/rupakroy/sonarcsv

**Libraries**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score ,r2_score ,confusion_matrix
from sklearn.metrics import confusion_matrix


%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

**LoadingData**

In [ ]:
sonar_data = pd.read_csv('/kaggle/input/connectionist-bench-sonar-mines-vs-rocks/sonar.all-data.csv', header=None)
dataSonarCopy=sonar_data.copy()

**Deleting Fist ROw  Cuz it is just a string**

In [ ]:
sonar_data = sonar_data.drop(index=0).reset_index(drop=True)


**Renaming the data**

In [ ]:
column_names = [f'freqi-{i}' for i in range(60)] + ['target']
sonar_data.columns = column_names


In [ ]:
sonar_data=pd.DataFrame(sonar_data)

**ExploringData**

In [ ]:
sonar_data.head()

In [ ]:
sonar_data.shape # (209 rows, 61 columns)

In [ ]:
sonar_data.info()

In [ ]:
sonar_data.value_counts().sum()

**ValueCountsOflables.**

In [ ]:
sonar_data['target'].value_counts()

In [ ]:
sonar_data.describe().T 

In [ ]:
sonar_data.isnull().sum()*100/sonar_data.shape[0] # = (0*100)/(545) = (0.0)
#Finally, you are multiplying by 100 and dividing by the total number of rows to get the percentage of missing values for each column.
#This expression provides insights into the percentage of missing data in each column of your 'sonarDataSet' dataset.

**Deleting first row cuz it is not important**

In [ ]:
sonar_data.head()

**Normalizing data by the minmax Algorithm Scales the data to a fixed range, typically between 0 and 1.**

In [ ]:
scaler = MinMaxScaler()
sonar_data.iloc[:, :-1] = scaler.fit_transform(sonar_data.iloc[:, :-1])  # Exclude the target column


In [ ]:
sonar_data.head()

** standard ScalesScales the data so that it has a mean of 0 and a standard deviation of 1
** 


In [ ]:
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# sonar_data.iloc[:, :-1] = sonar_data.fit_transform(data.iloc[:, :-1])  # Exclude the target column


In [ ]:
#sonar_data.head() that was standerd scaler but it was not the best choice for me 

**Visualization**

In [ ]:
sns.pairplot(sonar_data.iloc[:, :-1])
plt.suptitle('Pairplot of Features (Excluding Target Variable)', y=1.02)
plt.show()

**CorrelationOrRelationBetweenObjects**

In [ ]:
correlation_matrix = sonar_data.iloc[:, :-1].corr()
plt.figure(figsize=(10, 8))  # Adjust the figure size as needed
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5, fmt='.2f')

# Add title to the heatmap
plt.title('Correlation Matrix Heatmap')
plt.show()


**featureSelection Method**

 Recursive Feature Elimination (RFE) is a feature selection method that recursively removes features, building a model on the remaining features, and then ranking the importance of features based on their contribution to the model. Here's how you can apply RFE to your sonar sensor data:

In [ ]:
# Assuming 'sonar_data' is your DataFrame and the target column is named 'target'
X = sonar_data.drop('target', axis=1)  # Features
y = sonar_data['target']  # Target variable

# Create a RandomForestClassifier as the estimator
estimator = RandomForestClassifier()

# Create RFE selector
selector = RFE(estimator, n_features_to_select=50, step=1)

# Fit RFE selector
selector = selector.fit(X, y)

# Get selected features
selected_features = X.columns[selector.support_]

# Create new DataFrame with selected features
selected_data = sonar_data[selected_features]


**featureExtraction Method**

Principal Component Analysis (PCA) is a popular technique for feature extraction and dimensionality reduction. Here's how you can apply PCA to your data:

In [ ]:
#no the X columns become 50 with the feacture selection
# Assuming 'sonar_data' is your DataFrame and the target column is named 'target'
X = sonar_data.drop('target', axis=1)  # Features 
y = sonar_data['target']  # Target variable

# Initialize PCA with the number of components you want to keep
n_components = 40  Adjust the number of components as needed
pca = PCA(n_components=n_components)

# Fit PCA to the data
X_pca = pca.fit_transform(X)

# Create a DataFrame for the transformed features
column_names = [f" PC_freq{i+1}" for i in range(n_components)]  # PC1, PC2, ..., PCn
X_pca_df = pd.DataFrame(data=X_pca, columns=column_names)

# Concatenate the transformed features DataFrame with the target variable
sonar_data = pd.concat([X_pca_df, y], axis=1)


**SplittingData**

In [ ]:
X = sonar_data.drop(columns='target', axis=1)
Y = sonar_data['target']

**Droping First row cuz it just a string**

In [ ]:
# I have alraedy done with setp early
# X=X.drop([0])
# Y=Y.drop([0])

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25,  random_state=1)
#https://www.kaggle.com/code/tanavbajaj/sonar-rock-vs-mine-classifier

**Intailizating The model**

In [ ]:
model = LogisticRegression()


**Training The model**

In [ ]:
model.fit(X_train, Y_train)


**Predicting The Model**

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
print(y_pedict)

**Accuracy,Evaluation and Metrices**

In [ ]:
accuracy = accuracy_score(y_pred, y_test) 

In [ ]:
print('Accuracy on training data : ', training_data_accuracy)

# The accuracy will be 80% and nearby

**ConfusionMatrix**

In [ ]:
conf_matrix = confusion_matrix(y_test, y_pred)

print("Confusion Matrix:")
print(conf_matrix)

In [ ]:
# Plot confusion matrix heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, cmap='Blues', fmt='g', 
            xticklabels=['Predicted Negative', 'Predicted Positive'], 
            yticklabels=['Actual Negative', 'Actual Positive'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

**Try_A_Realistic_CASE**

In [ ]:
#60 one 
input_data = (0.0307,0.0523,0.0653,0.0521,0.0611,0.0577,0.0665,0.0664,0.1460,0.2792,0.3877,0.4992,0.4981,0.4972,0.5607,0.7339,0.8230,0.9173,0.9975,0.9911,0.8240,0.6498,0.5980,0.4862,0.3150,0.1543,0.0989,0.0284,0.1008,0.2636,0.2694,0.2930,0.2925,0.3998,0.3660,0.3172,0.4609,0.4374,0.1820,0.3376,0.6202,0.4448,0.1863,0.1420,0.0589,0.0576,0.0672,0.0269,0.0245,0.0190,0.0063,0.0321,0.0189,0.0137,0.0277,0.0152,0.0052,0.0121,0.0124,0.0055)
print(len(input_data))  # Output: 50


In [ ]:
# empty cell 

**Converting it into Array**

reshape the np array as we are predicting for one instance


In [ ]:
input_data_as_numpy_array_and_resahped = np.asarray(input_data).reshape(1,-1)


In [ ]:
# Reduce the input data to 40 elements
input_data_40 = input_data[:40]

# Apply MinMax scaling
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(np.array(input_data).reshape(-1, 1))

print(scaled_data)

In [ ]:
prediction = model.predict(input_data_as_numpy_array_and_resahped)
#prediction = model.predict(scaled_data)

print("the target Prediction from target " ,prediction)
if (prediction[0]=='R'):
  print('The object is a Rock')
else:
  print('The object is a mine')

# The output will be M means Mine

In [ ]:
# if (prediction[0]=='R'):
#   print('The object is a Rock')
# else:
#   print('The object is a mine')